<a href="https://colab.research.google.com/github/Nuhayd12/Coral_Classification_and_Prediction/blob/main/Coral_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#IMPORT LIBRARIES AND MODELS

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

#DATA PREPARATION


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, zoom_range=0.2)
test_val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Coral Clasification/Train', target_size=(150, 150), batch_size=32, class_mode='binary')
validation_generator = test_val_datagen.flow_from_directory('/content/drive/MyDrive/Coral Clasification/Valid', target_size=(150, 150), batch_size=32, class_mode='binary')
test_generator = test_val_datagen.flow_from_directory('/content/drive/MyDrive/Coral Clasification/Test', target_size=(150, 150), batch_size=32, class_mode='binary', shuffle=False)

Found 1522 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


# Model definition

In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

#Compilation

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Training

In [7]:
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])


Epoch 1/50
48/48 [==============================] - 340s 7s/step - loss: 0.4254 - accuracy: 0.8436 - val_loss: 1.2881 - val_accuracy: 0.6417
Epoch 2/50
48/48 [==============================] - 113s 2s/step - loss: 0.2227 - accuracy: 0.9152 - val_loss: 1.1759 - val_accuracy: 0.6833
Epoch 3/50
48/48 [==============================] - 108s 2s/step - loss: 0.2058 - accuracy: 0.9238 - val_loss: 0.9918 - val_accuracy: 0.7083
Epoch 4/50
48/48 [==============================] - 106s 2s/step - loss: 0.1832 - accuracy: 0.9323 - val_loss: 1.7127 - val_accuracy: 0.7000
Epoch 5/50
48/48 [==============================] - 107s 2s/step - loss: 0.1653 - accuracy: 0.9409 - val_loss: 1.4193 - val_accuracy: 0.7333
Epoch 6/50
48/48 [==============================] - 107s 2s/step - loss: 0.1737 - accuracy: 0.9448 - val_loss: 2.1132 - val_accuracy: 0.6583
Epoch 7/50
48/48 [==============================] - 103s 2s/step - loss: 0.1686 - accuracy: 0.9442 - val_loss: 2.2113 - val_accuracy: 0.7000
Epoch 8/50
48

#Evaluation

In [8]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


4/4 [==============================] - 33s 11s/step - loss: 0.3487 - accuracy: 0.9250
Test accuracy: 0.925000011920929


In [10]:
print(test_acc * 100,"%")

92.5000011920929 %


# Predictions and classification report

In [11]:
predictions = model.predict(test_generator)
y_true = test_generator.classes
y_pred = (predictions > 0.5).astype(int).flatten()

4/4 [==============================] - 4s 574ms/step


In [12]:
print(classification_report(y_true, y_pred, target_names=['Non-coral', 'Coral']))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

   Non-coral       0.87      1.00      0.93        60
       Coral       1.00      0.85      0.92        60

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.92       120
weighted avg       0.93      0.93      0.92       120

[[60  0]
 [ 9 51]]
